In [3]:
# download latest
import ftplib
from trialstreamer import config



homepage = "nlmpubs.nlm.nih.gov"


def get_ftp():
    """
    log in to FTP and pass back
    """
    ftp = ftplib.FTP(homepage)
    ftp.login(user="anonymous", passwd=config.PUBMED_USER_EMAIL)
    return ftp

ftp = get_ftp()

with open('data/pa2019.xml', 'wb') as f:
    ftp.retrbinary('RETR ' + 'online/mesh/MESH_FILES/xmlmesh/pa2019.xml', f.write)

In [60]:
import xml.etree.ElementTree as ET
tree = ET.parse('data/pa2019.xml')
root = tree.getroot()

In [61]:
pas = root.findall('PharmacologicalAction')

In [72]:
from collections import defaultdict

drugs_from_class = defaultdict(list)
class_from_drug = defaultdict(list)


for pa in pas:
    descriptor = pa.find('DescriptorReferredTo')
    substance_list = pa.find("PharmacologicalActionSubstanceList")
    
    d_ui = descriptor.find('DescriptorUI').text
    d_term = descriptor.find('DescriptorName/String').text
    
    
    for substance in substance_list:
        s_ui = substance.find('RecordUI').text
        s_term = substance.find('RecordName/String').text
        
        drugs_from_class[d_term].append({"mesh_ui": s_ui, "mesh_term": s_term})
        class_from_drug[s_term].append({"mesh_ui": d_ui, "mesh_term": d_term})
    

In [76]:
import pickle

with open('trialstreamer/data/drugs_from_class.pck', 'wb') as f:
    pickle.dump(drugs_from_class, f)
    
with open('trialstreamer/data/class_from_drug.pck', 'wb') as f:
    pickle.dump(class_from_drug, f)

<Element 'PharmacologicalActionSubstanceList' at 0x7f6d3ffbeef8>